In [1]:
print("ok")

ok


In [2]:
%pwd

'/Users/mariamkhan/health-bot/research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'/Users/mariamkhan/health-bot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
# Extract data from the PDF data file
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [7]:
# Return the extracted data
extracted_data = load_pdf_file(data='data/')

In [9]:
# View extracted data
# extracted_data

In [10]:
# Split data into text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks = text_split(extracted_data)
print("Length of text chunks: ", len(text_chunks))

Length of text chunks:  5860


In [12]:
from langchain_huggingface import HuggingFaceEmbeddings

In [13]:
# Download embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [15]:
embeddings = download_hugging_face_embeddings()

In [37]:
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [16]:
query_result = embeddings.embed_query("Hello World!")
print("Length", len(query_result))

Length 384


In [18]:
# query_result

In [19]:
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [22]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "health-bot"

pc.create_index (
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [23]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [24]:
# Embed each chunk and upsert the embeddings into Pinecone Index
from langchain.vectorstores import Pinecone as PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [43]:
# Load existing index 
# from langchain.vectorstores import Pinecone as PineconeVectorStore

# docsearch = PineconeVectorStore.from_existing_index(
    # index_name="health-bot",
    # embedding=embeddings
# )


In [25]:
docsearch

In [26]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [27]:
retrieved_docs = retriever.invoke("What is acne?")

In [28]:
retrieved_docs

[Document(metadata={'page': 39.0, 'source': 'data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(metadata={'page': 38.0, 'source': 'data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(metadata={'page': 37.0, 'source': 'data/Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteria.\nDescription\nAcne vulgaris, the medical term for c

In [29]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.4, max_tokens=500)

In [30]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are a question-answering assistant. "
    "Once the user provides a prompt or question, please use the pieces of retrieved context to answer the question. "
    "If you are not certain on an answer, just say you don't know. Do not make up random answers. "
    "Your answer must be 4 sentences max. Make it short and concise."
    "If asked about how to treat a condition or illness, give exact information. You can include steps on how to get better."
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

In [31]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [32]:
response = rag_chain.invoke({"input" : "What causes skin to rash?"})
print(response["answer"])



System: Skin can rash for a variety of reasons, including allergic reactions, exposure to irritants, and skin diseases like eczema and psoriasis. Rashes can also be caused by insect bites and stings, as well as viral skin diseases. If you have a rash that does not go away after seven days of treatment or is accompanied by other symptoms, it is important to consult a doctor for proper diagnosis and treatment.
